In [33]:
import os

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import fsspec
from tqdm.autonotebook import tqdm
import rioxarray
# import hydroeval as he
# import xesmf as xe
from xclim.sdba.adjustment import EmpiricalQuantileMapping, DetrendedQuantileMapping
import dask
import warnings
warnings.filterwarnings("ignore")

In [34]:
# get CMIP6 model table
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df.head()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,ps,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
1,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rsds,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
2,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rlus,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
3,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rlds,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
4,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,psl,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706


In [35]:
df_ro = df[(df.table_id == 'day') & (df.variable_id == 'mrro')]
len(df_ro)

632

In [36]:
def load_data(df, source_id, expt_id, grid_label):
    """
    Load 3hr runoff data for given source and expt ids
    """
    uri = df[(df.source_id == source_id) &
                         (df.experiment_id == expt_id) & (df.grid_label==grid_label)].zstore.values[0]

    ds = xr.open_zarr(fsspec.get_mapper(uri), consolidated=True).convert_calendar('standard', missing=0)
    return ds

In [43]:
# df_ro= df_ro[(df_ro.experiment_id.str.contains('historical')) | (df_ro.experiment_id.str.contains('ssp126')) | (df_ro.experiment_id.str.contains('ssp245')) | (df_ro.experiment_id.str.contains('ssp370'))|
#             (df_ro.experiment_id.str.contains('ssp585'))]
df_ro =  df[(df.experiment_id.str.contains('ssp126')) & (df.table_id == 'day') & (df.variable_id == 'mrro') & (df.activity_id=='ScenarioMIP')]
len(df_ro)

67

In [38]:
def swap_western_hemisphere(array):
    """Set longitude values in range -180, 180.
    Western hemisphere longitudes should be negative.
    """

    # Set longitude values in range -180, 180.
    array['lon'] = (array['lon'] + 180) % 360 - 180

    # Re-index data along longitude values
    west = array.where(array.lon < 0, drop=True)
    east = array.where(array.lon >= 0, drop=True)
    return west.combine_first(east)

In [39]:
def regrid_to_1deg(array):
    ds_out = xr.Dataset({'lat': (['lat'], np.arange(89.5, -89.5, -1)),
                     'lon': (['lon'], np.arange(-179.5, 179.5, 1))})
    regridder = xe.Regridder(array, ds_out, 'bilinear')
    # regridder.clean_weight_file()
    out= regridder(array) 
    return out

In [44]:
def download_data(df_ro, source_id, scenario, grid):
    ds= load_data(df_ro, source_id, scenario, grid).sel(time=slice('1950-01-01', '2015-01-01'))['mrro'].convert_calendar('standard',
                                                            missing=0, use_cftime=True).to_dataset()
    ds= swap_western_hemisphere(ds)
    # ds= regrid_to_1deg(ds)
    (ds*86400).assign_attrs(units='mm/day').compute().to_netcdf('%s_hist_%s_1x1deg.nc'%(source_id.replace('-','_'), grid),
                        encoding = {"mrro": {'zlib': True, 'dtype':'float32', '_FillValue':-9999}})

In [45]:
df_ro

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
15482,ScenarioMIP,NOAA-GFDL,GFDL-ESM4,ssp126,r1i1p1f1,day,mrro,gr1,gs://cmip6/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-ES...,NaN,20180701
56549,ScenarioMIP,CCCma,CanESM5,ssp126,r1i1p1f1,day,mrro,gn,gs://cmip6/CMIP6/ScenarioMIP/CCCma/CanESM5/ssp...,NaN,20190306
56560,ScenarioMIP,CCCma,CanESM5,ssp126,r7i1p1f1,day,mrro,gn,gs://cmip6/CMIP6/ScenarioMIP/CCCma/CanESM5/ssp...,NaN,20190306
56840,ScenarioMIP,CCCma,CanESM5,ssp126,r9i1p1f1,day,mrro,gn,gs://cmip6/CMIP6/ScenarioMIP/CCCma/CanESM5/ssp...,NaN,20190306
57091,ScenarioMIP,CCCma,CanESM5,ssp126,r5i1p1f1,day,mrro,gn,gs://cmip6/CMIP6/ScenarioMIP/CCCma/CanESM5/ssp...,NaN,20190306
...,...,...,...,...,...,...,...,...,...,...,...
360267,ScenarioMIP,NIMS-KMA,KACE-1-0-G,ssp126,r1i1p1f1,day,mrro,gr,gs://cmip6/CMIP6/ScenarioMIP/NIMS-KMA/KACE-1-0...,NaN,20191011
363915,ScenarioMIP,MIROC,MIROC6,ssp126,r2i1p1f1,day,mrro,gn,gs://cmip6/CMIP6/ScenarioMIP/MIROC/MIROC6/ssp1...,NaN,20191016
364300,ScenarioMIP,MIROC,MIROC6,ssp126,r1i1p1f1,day,mrro,gn,gs://cmip6/CMIP6/ScenarioMIP/MIROC/MIROC6/ssp1...,NaN,20191016
364597,ScenarioMIP,MIROC,MIROC6,ssp126,r3i1p1f1,day,mrro,gn,gs://cmip6/CMIP6/ScenarioMIP/MIROC/MIROC6/ssp1...,NaN,20191016


In [15]:
ds= load_data(df_ro, source_id, scenario, grid)['mrro'].convert_calendar('standard',
                                                            missing=0, use_cftime=True).to_dataset()

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 180, lon: 288, time: 31411)
Coordinates:
  * bnds       (bnds) float64 1.0 2.0
  * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lon        (lon) float64 0.625 1.875 3.125 4.375 ... 355.6 356.9 358.1 359.4
  * time       (time) datetime64[ns] 2015-01-01T12:00:00 ... 2100-12-31T12:00:00
    lat_bnds   (lat, bnds) float64 dask.array<chunksize=(180, 2), meta=np.ndarray>
    lon_bnds   (lon, bnds) float64 dask.array<chunksize=(288, 2), meta=np.ndarray>
    time_bnds  (time, bnds) object dask.array<chunksize=(15706, 2), meta=np.ndarray>
Data variables:
    mrro       (time, lat, lon) float32 dask.array<chunksize=(965, 180, 288), meta=np.ndarray>
Attributes: (12/49)
    Conventions:            CF-1.7 CMIP-6.0 UGRID-1.0
    activity_id:            ScenarioMIP
    branch_method:          standard
    branch_time_in_child:   60225.0
    branch_time_in_parent:  60225.0
    comment:                <null ref>
    ...                     ...
    tracking_id:            hdl:21.14100/0693a5c3-6ebe-4ec4-8cd6-b3f8d2436e90...
    variable_id:            mrro
    variant_info:           N/A
    variant_label:          r1i1p1f1
    netcdf_tracking_ids:    hdl:21.14100/0693a5c3-6ebe-4ec4-8cd6-b3f8d2436e90...
    version_id:             v20180701

In [11]:
df_ro

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
16024,ScenarioMIP,NOAA-GFDL,GFDL-ESM4,ssp119,r1i1p1f1,day,mrro,gr1,gs://cmip6/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-ES...,NaN,20180701
56996,ScenarioMIP,CCCma,CanESM5,ssp119,r1i1p1f1,day,mrro,gn,gs://cmip6/CMIP6/ScenarioMIP/CCCma/CanESM5/ssp...,NaN,20190306
71978,ScenarioMIP,IPSL,IPSL-CM6A-LR,ssp119,r1i1p1f1,day,mrro,gr,gs://cmip6/CMIP6/ScenarioMIP/IPSL/IPSL-CM6A-LR...,NaN,20190410
205215,ScenarioMIP,MRI,MRI-ESM2-0,ssp119,r1i1p1f1,day,mrro,gn,gs://cmip6/CMIP6/ScenarioMIP/MRI/MRI-ESM2-0/ss...,NaN,20190603
